## <span style="font-family:Copperplate Gothic"><u>Mask Detection Using TensorFlow</u></span> 


> ``Analyst Profile Information``
     
| Data Analyst Profile Information    :|                            | 
|---                     |---                                 |
|Person Name  →          |<b>Tejas Buradkar</b>             |
|E-Mail       →          |buradkartejas5@gmail.com           |
|Reference Name →        |IABAC:-International Association of Business Analytics Certification    |
|Module type →           |Certified Data Scientist - Project  |


In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
# initialize the initial learning rate, number of epochs to train for,
# and batch size
# LR learning rate keeping low LR brings more accuracy
INIT_LR = 1e-4
EPOCHS = 20
# batch size
BS = 32

In [3]:
DIRECTORY = "./data/"
CATEGORIES = ["with_mask", "without_mask"]

In [4]:
DIRECTORY

'./data/'

In [5]:
# grab the list of images in our dataset directory, then initialize
# the list of data (i.e., images) and class images
print("[INFO] loading images...")

data = []
labels = []


[INFO] loading images...


In [6]:

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        image = preprocess_input(image)

        data.append(image)
        labels.append(category)

In [7]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

data = np.array(data, dtype="float32")
labels = np.array(labels)

In [8]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.20, stratify=labels, random_state=42)

# construct the training image generator for data augmentation
# multipying single image and making more outof single
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [9]:
# load the MobileNetV2 network, ensuring the head FC layer sets are
# left off
baseModel = MobileNetV2(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

9406464/9406464 [==============================] - 4s 0us/step


In [10]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

In [11]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [12]:
# compile our model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


In [13]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=BS),
	steps_per_epoch=len(trainX) // BS,
	validation_data=(testX, testY),
	validation_steps=len(testX) // BS,
	epochs=EPOCHS)

[INFO] training head...
Epoch 1/20
102/102 [==============================] - 114s 1s/step - loss: 0.3612 - accuracy: 0.8816 - val_loss: 0.1419 - val_accuracy: 0.9792
Epoch 2/20
102/102 [==============================] - 141s 1s/step - loss: 0.1433 - accuracy: 0.9661 - val_loss: 0.0825 - val_accuracy: 0.9853
Epoch 3/20
102/102 [==============================] - 134s 1s/step - loss: 0.1016 - accuracy: 0.9735 - val_loss: 0.0630 - val_accuracy: 0.9866
Epoch 4/20
102/102 [==============================] - 123s 1s/step - loss: 0.0833 - accuracy: 0.9760 - val_loss: 0.0559 - val_accuracy: 0.9866
Epoch 5/20
102/102 [==============================] - 124s 1s/step - loss: 0.0689 - accuracy: 0.9784 - val_loss: 0.0495 - val_accuracy: 0.9878
Epoch 6/20
102/102 [==============================] - 128s 1s/step - loss: 0.0611 - accuracy: 0.9830 - val_loss: 0.0461 - val_accuracy: 0.9890
Epoch 7/20
102/102 [==============================] - 124s 1s/step - loss: 0.0589 - accuracy: 0.9815 - val_loss: 0.046

In [14]:
# make predictions on the testing set
print("[INFO] evaluating network...")
predIdxs = model.predict(testX, batch_size=BS)

[INFO] evaluating network...
26/26 [==============================] - 18s 632ms/step


In [15]:
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

In [16]:
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       433
without_mask       0.99      0.99      0.99       386

    accuracy                           0.99       819
   macro avg       0.99      0.99      0.99       819
weighted avg       0.99      0.99      0.99       819



In [18]:
# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("mask_detector1.model", save_format="h5")

[INFO] saving mask detector model...
